## Converting classifier to detector


### Learning and code taken from Adrian's Tutorial on Object detcetion :
https://pyimagesearch.com/2020/07/13/r-cnn-object-detection-with-keras-tensorflow-and-deep-learning/

### 1. Using sliding window 
####  a.Use sliding window to get the ROI (location) and feed that into your CNN classifier
####  b. No change ine the deep neural network
####   c. Need to implement a preprocessing step

In [1]:
import os
import cv2
import time
import numpy as np

In [2]:
def sliding_window(image, step, ws):
    # slide a window across the image
    for y in range(0, image.shape[0] - ws[1], step):
        for x in range(0, image.shape[1] - ws[0], step):
            # yield the current window
            yield (x, y, image[y:y + ws[1], x:x + ws[0]])

In [3]:
# step : stride for sliding the window across image (recommendation is 4 to 8 pixels)
# ws: The ROI or window size for cropping out the image and feeding to the CNN 

In [4]:
base_path=r"D:\LearningObjectDetection\images"
image_path = os.path.join(base_path,"dog1.jpg")
img = cv2.imread(image_path)
print(img.shape)
cv2.imshow("img", img)
cv2.waitKey(0)
#cv2.destroyAllWindows()


(360, 479, 3)


-1

In [5]:
step=16
window_size=(224,224)
img_resize_width=600

In [ ]:
# initialize variables used for the object detection procedure
WIDTH = 600
PYR_SCALE = 1.5
WIN_STEP = 16
#ROI_SIZE = eval(args["size"])
INPUT_SIZE = (224, 224)

In [ ]:
for x,y,img_crop in sliding_window(img, step, window_size):
    print(x,y)
    img_vis=img.copy()
    cv2.rectangle(img_vis, (x, y), (x+window_size[0], y+window_size[0]), (255,0,0), 2)
    cv2.imshow("img", img_vis)
    cv2.waitKey(30)
cv2.destroAllWindows()

In [ ]:
### importing the resnet model
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.applications import imagenet_utils
from imutils.object_detection import non_max_suppression
import imutils

In [ ]:
model_input_size=(224,224)

In [ ]:
print("[INFO] loading network...")
model = ResNet50(weights="imagenet", include_top=True)
# load the input image from disk, resize it such that it has the
# has the supplied width, and then grab its dimensions
img = cv2.imread(image_path)
print("[INFO] image shape: ",img.shape)
img = cv2.resize(img, model_input_size)
#img = imutils.resize(img, width=img_resize_width)
print("[INFO] image shape after resize: ",img.shape)
img = img_to_array(img)
img = preprocess_input(img)
img = np.expand_dims(img,axis=0)
print("[INFO] image shape after preprocessing: ",img.shape)


#(H, W) = img.shape[:2]

In [ ]:
print("[INFO] classifying the image...")
start = time.time()
preds = model.predict(img)
end = time.time()
print("[INFO] classifying image took {:.5f} seconds".format(end - start))
print("[INFO] Prediction values: ")
print("num prediction: ",len(preds[0]))
#print(preds)
print("max value and class number: ",np.max(preds[0]),"and",np.argmax(preds[0]))
# decode the predictions and initialize a dictionary which maps class
# labels (keys) to any ROIs associated with that label (values)
preds = imagenet_utils.decode_predictions(preds, top=1)
print ("prediction decoded: Imagenet ID, Class Name, Class Prob")
print(preds)
labels = {}

### Using Sliding window and ResNet50 for object detection
### Many of the ROIs will be simply background so wee need to filter out those based on threshold probbility score

In [ ]:
#min_conf=0.6
min_conf=0.8

In [ ]:
# loop over the sliding window locations and store each region and its location
rois=[]
locs=[]
img = cv2.imread(image_path)
for (x, y, roi_cropped) in sliding_window(img,  step, window_size):

    w=window_size[0]
    h=window_size[1]
    # take the ROI and preprocess it so we can later classify
    # the region using Keras/TensorFlow
    roi = cv2.resize(roi_cropped, model_input_size)
    roi = img_to_array(roi)
    roi = preprocess_input(roi)
    # update our list of ROIs and associated coordinates
    rois.append(roi)
    locs.append((x, y, x+w, y+h))
    
    img_vis=img.copy()
    img_roi=roi_cropped.copy()
    cv2.rectangle(img_vis, (x, y), (x+w, y+h), (255,0,0), 2)
    cv2.imshow("img", img_vis)
    cv2.imshow("cropped_roi", img_roi)
    cv2.waitKey(30)
cv2.destroyAllWindows()

#### Do the prediction on all rois as a batch prediction

In [ ]:
# convert the ROIs to a NumPy array
rois = np.array(rois, dtype="float32")
# classify each of the proposal ROIs using ResNet and then show how
# long the classifications took
print("[INFO] classifying ROIs...")
start = time.time()
preds = model.predict(rois)
end = time.time()
print("[INFO] classifying {} ROIs took {:.5f} seconds".format(rois.shape[0],end - start))
# decode the predictions and initialize a dictionary which maps class
# labels (keys) to any ROIs associated with that label (values)
preds = imagenet_utils.decode_predictions(preds, top=1)

In [ ]:
print(preds)

In [ ]:
labels = {}
# loop over the predictions
for (i, p) in enumerate(preds):
	# grab the prediction information for the current ROI
	(imagenetID, label, prob) = p[0]
	# filter out weak detections by ensuring the predicted probability
	# is greater than the minimum probability
	if prob >= min_conf:
		# grab the bounding box associated with the prediction and
		# convert the coordinates
		box = locs[i]
		# grab the list of predictions for the label and add the
		# bounding box and probability to the list
		L = labels.get(label, [])
		L.append((box, prob))
		labels[label] = L

In [ ]:
print(label)
L = labels.get(label, [])
print(L)

In [ ]:
labels

In [ ]:
# loop over the labels for each of detected objects in the image
img_all_preds = img.copy()
for label in labels.keys():
    # clone the original image so that we can draw on it
    print("[INFO] showing results for '{}'".format(label))
    clone = img.copy()
    # loop over all bounding boxes for the current label
    for (box, prob) in labels[label]:
        # draw the bounding box on the image
        (startX, startY, endX, endY) = box
        cv2.rectangle(clone, (startX, startY), (endX, endY),(0, 255, 0), 2)
        cv2.rectangle(img_all_preds, (startX, startY), (endX, endY),(0, 255, 0), 2)
    cv2.imshow("{}".format(label), clone)
    #cv2.waitKey(0)
    #clone = orig.copy()
cv2.imshow("Before", img_all_preds)
cv2.waitKey(0)

In [ ]:
### Applying NMS to get rid of boxes

In [ ]:
# loop over the labels for each of detected objects in the image
img_all_preds_bf=img.copy()
img_all_preds_af=img.copy()

for label in labels.keys():
    # clone the original image so that we can draw on it
    print("[INFO] showing results for '{}'".format(label))
    clone = img.copy()
    # loop over all bounding boxes for the current label
    for (box, prob) in labels[label]:
        # draw the bounding box on the image
        (startX, startY, endX, endY) = box
        cv2.rectangle(clone, (startX, startY), (endX, endY),(0, 255, 0), 2)
        cv2.rectangle(img_all_preds_bf, (startX, startY), (endX, endY),(0, 255, 0), 2)
    cv2.imshow("{} Before".format(label), clone)
    
    # extract the bounding boxes and associated prediction robabilities, then apply non-maxima suppression
    boxes = np.array([p[0] for p in labels[label]])
    proba = np.array([p[1] for p in labels[label]])
    boxes = non_max_suppression(boxes, proba,overlapThresh=0.2)
    # loop over all bounding boxes that were kept after applying
    # non-maxima suppression
    clone = img.copy()
    for (startX, startY, endX, endY) in boxes:
        # draw the bounding box and label on the image
        cv2.rectangle(clone, (startX, startY), (endX, endY),(0, 255, 0), 2)
        y = startY - 10 if startY - 10 > 10 else startY + 10
        cv2.putText(clone, label, (startX, y),cv2.FONT_HERSHEY_SIMPLEX, 0.45, (0, 255, 0), 2)
        cv2.putText(clone, str(endX-startx), (startX, y),cv2.FONT_HERSHEY_SIMPLEX, 0.45, (0, 255, 0), 2)

        cv2.rectangle(img_all_preds_af, (startX, startY), (endX, endY),(0, 255, 0), 2)
    # show the output after apply non-maxima suppression
    cv2.imshow("{} After".format(label), clone)

    #cv2.waitKey(0)
    #clone = orig.copy()
cv2.imshow("Before NMS", img_all_preds_bf)
cv2.imshow("After NMS", img_all_preds_af)

cv2.waitKey(0)

#### we see false positives here whihc we can reduce by increasing the min confidence

## Image Pyramid

#### Utilizing an image pyramid allows us to find objects in images at different scales (sizes) of an image. And when combined with a sliding window we can find objects in images in various locations.
#### At the bottom of the pyramid, we have the original image at its original size (in terms of width and height). And at each subsequent layer, the image is resized (subsampled) and optionally smoothed (usually via Gaussian blurring). The image is progressively subsampled until some stopping criterion is met, which is normally when a minimum size has been reached and no further subsampling needs to take place.
#### REf: https://pyimagesearch.com/2015/03/16/image-pyramids-with-python-and-opencv/

#### Combined with image pyramids, sliding windows allow us to localize objects at different locations and multiple scales of the input image:

### We need to define two parameters
### scale: by what scale images are subsample
### minimum size of the image for stopping criteria

In [ ]:
### we are going to run teh sliding window for each scale of the image

In [ ]:
def image_pyramid(image, scale=1.5, minSize=(224, 224)):
	# yield the original image
	yield image
	# keep looping over the image pyramid
	while True:
		# compute the dimensions of the next image in the pyramid
		w = int(image.shape[1] / scale)
		image = imutils.resize(image, width=w)
		# if the resized image does not meet the supplied minimum
		# size, then stop constructing the pyramid
		if image.shape[0] < minSize[1] or image.shape[1] < minSize[0]:
			break
		# yield the next image in the pyramid
		yield image

In [ ]:
# initialize variables used for the object detection procedure
base_img_width = 600
pyr_scale = 1.5
min_size=(224,224)

In [ ]:
img = cv2.imread(image_path)
print("[INFO] image shape: ",img.shape)
img = imutils.resize(img, width=img_resize_width)
img_h,img_w=img.shape[:2]
print("[INFO] image shape after resize: ",img.shape)


In [ ]:
for img_pr in image_pyramid(img,scale=1.5,minSize=min_size):
    print(img_pr.shape)
    cv2.imshow("img_pr",img_pr)
    cv2.waitKey(0)

### Combine image pyramid with sliding window

In [ ]:
roi_size=(224,224)

In [ ]:
# initialize the image pyramid
pyramid = image_pyramid(img, scale=pyr_scale, minSize=roi_size)
# initialize two lists, one to hold the ROIs generated from the image
# pyramid and sliding window, and another list used to store the
# (x, y)-coordinates of where the ROI was in the original image
rois = []
locs = []
# time how long it takes to loop over the image pyramid layers and
# sliding window locations
start = time.time()
# loop over the image pyramid
for image in pyramid:
    # determine the scale factor between the *original* image
    # dimensions and the *current* layer of the pyramid
    scale = img_w / float(image.shape[1])
    # for each layer of the image pyramid, loop over the sliding
    # window locations
    for (x, y, roi_cropped) in sliding_window(image, step, roi_size):
        # scale the (x, y)-coordinates of the ROI with respect to the
        # *original* image dimensions
        org_x,org_y,org_w,org_h=x,y,roi_size[0],roi_size[1]
        print("org: ",org_x,org_y,org_w,org_h)
        x = int(x * scale)
        y = int(y * scale)
        w = int(roi_size[0] * scale)
        h = int(roi_size[1] * scale)
        print("scaled: ", x,y,w,h)
        # take the ROI and preprocess it so we can later classify
        # the region using Keras/TensorFlow
        roi = cv2.resize(roi_cropped, model_input_size)
        roi = img_to_array(roi)
        roi = preprocess_input(roi)
        # update our list of ROIs and associated coordinates
        rois.append(roi)
        locs.append((x, y, x + w, y + h))
        
        img_vis=image.copy()
        img_roi=roi_cropped.copy()
        cv2.rectangle(img_vis, (x, y), (x+w, y+h), (255,0,0), 2)
        cv2.imshow("img", img_vis)
        cv2.imshow("cropped_roi", img_roi)
        cv2.waitKey(30)
cv2.destroyAllWindows()

In [ ]:
# blue scaled
# green original
img_test=image.copy()
cv2.rectangle(img_test, (x, y), (x+w, y+h), (255,0,0), 2)
cv2.rectangle(img_test, (org_x, org_y), (org_x+org_w, org_y+org_h), (0,255,0), 2)

img_test_org=img.copy()
cv2.rectangle(img_test_org, (x, y), (x+w, y+h), (255,0,0), 2)
cv2.rectangle(img_test_org, (org_x, org_y), (org_x+org_w, org_y+org_h), (0,255,0), 2)

cv2.imshow("img_resized", img_test)
cv2.imshow("img_original", img_test_org)

cv2.waitKey(0)

In [ ]:
img_test_org=img.copy()
cv2.rectangle(img_test_org, (x, y), (x+w, y+h), (255,0,0), 2)

cv2.rectangle(img_test_org, (org_x, org_y), (org_x+org_w, org_y+org_h), (0,255,0), 2)
cv2.imshow("img2", img_test_org)
cv2.waitKey(0)

In [ ]:
# crop with the original coord from resized image
# crop with the scaled coord from original image

In [ ]:
img_resized_crop=image[org_y:org_y + org_w, org_x:org_x + org_h]
img_org_crop=img[y:y + w, x:x + h]
img_org_crop_org_coord=img[org_y:org_y + org_w, org_x:org_x + org_h]
cv2.imshow("img_resized_crop", img_resized_crop)
cv2.imshow("img_org_crop", img_org_crop)
cv2.imshow("img_org_crop_org_coord", img_org_crop_org_coord)

cv2.waitKey(0)

### Drawbacks of the above approach
### 1. It is a very slow approcah to slide windows and if we add image pyramid, it further slows. 
### 2. Very tough to estimate the window size and stride, This however determines the accuracy of the object detector.
### 3. This is not end to end trainable, whihc means we cannot correct the error on the boundin box location